In [10]:
import tkinter as tk
from tkinter import filedialog as fd
import cv2 as cv
import numpy as np
from PIL import Image, ImageTk
import os
import openpyxl
import face_recognition

In [11]:
class Canvas():
    
    def __init__(self,window):
        self.window = window
        self.canvas1 = tk.Canvas(self.window, width=1200, height=800, bg='white')
        self.canvas2 = tk.Canvas(self.window, width=340, height=495, bg='white')
        self.canvas3 = tk.Canvas(self.window, width=340, height=300, bg='white')
        self.canvas1.create_rectangle(2, 3, 1195, 795, outline='black', width=2)
        self.canvas2.create_rectangle(5,3,330,490,outline='black',width=2)
        self.canvas3.create_rectangle(5,3,330,295,outline='black',width=2)
        self.canvas2.create_text(150,15,text="List",font=("Arial", 12))
        self.canvas1.create_text(600,25,text="NHẬN DIỆN KHUÔN MẶT",font=("Arial", 25))
        self.canvas1.place(x=0,y=0)
        self.canvas2.place(x=1200,y=0)
        self.canvas3.place(x=1200,y=500)
        self.canvas1.create_line(5, 50, 1194, 50, fill="black", width=2)
        self.canvas3.create_line(7, 150, 328, 150, fill="black", width=2)
        self.canvas3.create_text(160,135,text="Add Person",font=("Arial", 12))
    def __init__1(self):
        self.canvas1 = tk.Canvas(self.window, width=1200, height=800, bg='white')
        self.canvas1.create_rectangle(2, 3, 1195, 795, outline='black', width=2)
        self.canvas1.create_text(600,25,text="NHẬN DIỆN KHUÔN MẶT",font=("Arial", 25))
        self.canvas1.place(x=0,y=0)
        self.canvas1.create_line(5, 50, 1194, 50, fill="black", width=2)

        self.canvas2 = tk.Canvas(self.window, width=340, height=495, bg='white')
        self.canvas2.create_rectangle(5,3,330,490,outline='black',width=2)
        self.canvas2.place(x=1200,y=0)
        self.canvas2.create_text(150,15,text="List",font=("Arial", 12))
        # self.canvas1.create_line(1202, 500, 1528, 500, fill="black", width=2)
    def clear_console(self):
        self.canvas1.delete('all')
        self.canvas2.delete('all')
        self.__init__1()
        
    def getCanvas1(self):
        return self.canvas1
    def getCanvas2(self):
        return self.canvas2

In [12]:
class show_image():
    def __init__(self,window,canvas2):
        self.path = "DataSet"
        self.all_name_path = []
        self.canvas2 =canvas2
        self.known_face_encodings = {}
        self.known_face_names = []
        self.window =window
        self.attendance=[]
    def load_data(self):
        # [N21DCCN067,N21DCCN069,N21DCCN075]
        self.all_name_path = [os.path.join(self.path,f) for f in os.listdir(self.path)]
        #[[],[]]
        name_paths = []
        for name_path in self.all_name_path:
            name_paths.append([os.path.join(name_path,f) for f in os.listdir(name_path)])
        self.all_name_path= name_paths
        self.unknown_image_path = fd.askopenfilename(filetypes=[("Image Files", "*.png;*.jpg;*.jpeg")])
        self.unknown_image = face_recognition.load_image_file(self.unknown_image_path)

        self.image = cv.imread(self.unknown_image_path)
        
        for image_path_list in self.all_name_path:
            image_encodings = []
            if(len(image_path_list)==0):continue
            for image_path in image_path_list:
                image = face_recognition.load_image_file(image_path)
                image_encoding = face_recognition.face_encodings(image)[0]
                image_encodings.append(image_encoding)
            #[[dataset\\N21dccn067\\N21dccn067(1).img]]
            name = image_path_list[0].split("\\")[-1].split('(')[0]
            #[p,q]
            self.known_face_names.append(name)
            #{p:[],q:[]}
            self.known_face_encodings[name] = image_encodings 
    def show_image(self):
        
        self.load_data()
        
        self.face_locations= face_recognition.face_locations(self.unknown_image)
        self.unknown_face_encodings = face_recognition.face_encodings(self.unknown_image,self.face_locations)

        for (top, right, bottom, left), unknown_face_encoding in zip(self.face_locations, self.unknown_face_encodings):
            # matches = face_recognition.compare_faces(self.known_face_encodings, unknown_face_encoding)
            name_match = None
            min = 0.5
            time_match = 0
            for list_face_encoding in self.known_face_encodings.items():
                dis = face_recognition.face_distance(list_face_encoding[1],unknown_face_encoding)
                if dis.min() < min:
                    min = dis.min()
                    name_match = list_face_encoding[0]
                if dis.min()<0.55:
                    time_match+=1
                
            name = "Unknown"
            # print(min)
            if min<=0.45 and time_match>=2:
                self.known_face_encodings.pop(name_match)
                self.attendance.append(name_match)
                name = name_match
            # Vẽ khung bao quanh khuôn mặt bằng OpenCV
            cv.rectangle(self.image, (left, top), (right, bottom), (0, 255, 0), 2)
            
            # Hiển thị tên dưới khung bao quanh khuôn mặt
            cv.putText(self.image, name, (left, bottom + 20), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        self.attendance.sort()
        List_Show(self.attendance,self.canvas2)
        write_file(self.attendance,"File\\a.xlsx")
        img = Image.fromarray(cv.cvtColor(self.image,cv.COLOR_BGR2RGB))
        img = img.resize((1183, 733), Image.LANCZOS)
        img=ImageTk.PhotoImage(img)
        panel = tk.Label(self.window, image=img)
        panel.image = img
        panel.place(x=4,y=55)
        cv.destroyAllWindows()

        

In [13]:
def List_Show(list , canvas2):
    y = 60
    for name in list:
        canvas2.create_text(60,y,text="- "+name,font=("Arial", 10))
        y+=15

In [14]:
class add_person():
    def __init__(self,window=None,canvas=None,name=None):
        if window==None:return
        self.cam = cv.VideoCapture(0)
        self.canvas= canvas.getCanvas1()
        self.clear = canvas.clear_console
        self.name=name
        self.image=None
        self.face = cv.CascadeClassifier("haarcascade_frontalface_default.xml")
        self.stop=0
        self.captured_button = tk.Button(window,text="OK",command=lambda: self.stop_fuc())
        self.captured_button.place(x=1350,y=540)
    def stop_fuc(self):self.stop=1
    def capture(self): 
        check,frame = self.cam.read()
        saved_frame=frame
        frame = cv.resize(frame,(1183, 733))
        if check:
            gray=cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
            faces = self.face.detectMultiScale(gray,scaleFactor=1.2,minNeighbors=5)
            for (x,y,w,h) in faces:
                cv.putText(frame,self.name, (x, y-10), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv.rectangle(frame,(x, y), (x+w, y+h), (0, 255, 0), 2)
            self.image = cv.cvtColor(frame, cv.COLOR_BGR2RGB) # to RGB
            self.image = Image.fromarray(self.image) # to PIL format
            self.image = ImageTk.PhotoImage(image=self.image) # to ImageTk format
            # Update image
            self.canvas.create_image(4,55,anchor = tk.NW, image= self.image)
            self.canvas.img_tk= self.image
            #check i to return
            if(self.stop):
                self.cam.release()
                self.clear()
                self.captured_button.destroy()
                self.save_image(self.name,saved_frame)
                return
            # Repeat every 'interval' ms
            self.canvas.after(10,self.capture)
        else :
            print("Error")
            return
    def save_image(self,name,image):
        all_name_path = [os.path.join("DataSet",f) for f in os.listdir("DataSet")]
        all_name = [x.split('\\')[-1] for x in all_name_path]
        if name not in all_name:
            os.mkdir("DataSet\\" + name)
        max = 0
        for x in os.listdir("DataSet\\"+name):
            x=x.split("(")[-1].split(")")[0]
            if int(x)>max:max = int(x)
        max +=1
        cv.imwrite("DataSet\\"+name + "\\"+name+"("+str(max)+")"+".jpg",image)

    def pick_picture(self):
        image_path = fd.askopenfilename(filetypes=[("Image Files", "*.png;*.jpg;*.jpeg")])
        image = cv.imread(image_path)
        self.save_image(image_path.split('/')[-1].split('.')[0],image)
        

In [15]:
def button2_clicked(window,entry,canvas):
    canvas.clear_console()
    name=entry.get()
    if name == "":return
    a = add_person(window,canvas,name)
    a.capture()


In [16]:
class interface():
    def __init__(self):
        self.window = tk.Tk()
        # setting attribute
        self.window.attributes('-fullscreen', True)
        self.window.title("Nhận diện người")

        
        self.canvas= Canvas(self.window)

        self.entry = tk.Entry(self.window)
        self.entry.place(x=1310,y=512)

        self.button0 = tk.Button(self.window,text= "From picture",command= lambda: add_person().pick_picture())
        self.button0.place(x=1330,y=580)
        
        self.button1 = tk.Button(self.window, text="Pick picture",command=lambda:show_image(self.window,self.canvas.getCanvas2()).show_image())
        self.button1.place(x=600,y=810)

        self.button2 = tk.Button(self.window,text="   Capture   ",command=lambda:button2_clicked(self.window,self.entry,self.canvas))
        self.button2.place(x=1220,y=510)

        self.button3 = tk.Button(self.window,text="    Clear    ",command=lambda: self.canvas.clear_console())
        self.button3.place(x=1350 ,y=700)

        self.button4 = tk.Button(self.window,text="    Close    ",command= lambda:self.close())
        self.button4.place(x=1350,y=765)

        # Chạy vòng lặp chính
        self.window.mainloop()
    def close(self):
        self.window.destroy()

In [17]:
def write_file(list_MSSV,file_path):

    # Đọc dữ liệu từ file Excel
    workbook = openpyxl.load_workbook(file_path)
    sheet = workbook.active

    data = []
    print(list_MSSV)
    for row in sheet.iter_rows(min_row=3, values_only=True):
        # print(row,end="\n")
        stt, MSSV, Ho, Ten, Lop, diem_danh,*empty_value = row
        # print(Ten)
        if MSSV in list_MSSV:
            diem_danh+=1
        data.append({"stt": stt, "Mã sinh viên": MSSV, "Họ": Ho, "Tên": Ten, "Lớp":Lop, "Điểm danh": diem_danh})
    # Tạo một workbook mới
    workbook = openpyxl.Workbook()
    sheet = workbook.active

    # Ghi tiêu đề cột
    sheet.append(["TT", "MSSV", "Họ", "tên sinh viên","Lớp","Điểm danh" ])

    # Ghi dữ liệu vào sheet
    for entry in data:
        sheet.append([entry["stt"], entry["Mã sinh viên"], entry["Họ"], entry["Tên"], entry["Lớp"],entry["Điểm danh"]])

    # Lưu workbook thành file Excel mới
    workbook.save(file_path)


In [18]:
if __name__== '__main__':
    interface()
    

['N21DCCN001', 'N21DCCN002', 'N21DCCN003', 'N21DCCN067 ']
